In [2]:
import math
class Vector:
    
    def __init__(self, values):
        self._length = len(values)
        self._values = values
        
    def __add__(self, other):
        if not isinstance(other, type(self)):
            raise Exception()
        if self._length != other._length:
            raise Exception("Cannot add objects of class Vector with different length")
        return Vector([self._values[i] + other._values[i] for i in range(self._length)])
    
    def __sub__(self, other):
        return self.__add__(-other)
    
    def scalarMultiply(self, scalar):
        if not isinstance(scalar, float) and not isinstance(scalar, int):
            raise Exception("Non-scalar multiplication not implemented")
        return Vector([scalar * v for v in self._values])
    
    def __neg__(self):
        return Vector([-x for x in self._values])
        
    def getLength(self):
        return self._length
        
    def getValues(self):
        return self._values
    
    def truncate(self, length):
        return Vector([self._values[i] for i in range(length)])
    
    def getAbsoluteValue(self):
        return math.sqrt(self.innerProduct(self))
    
    def innerProduct(self, other):
        if not isinstance(other, type(self)):
            raise Exception()
        if self._length != other._length:
            raise Exception()
        return sum([self._values[i] * other._values[i] for i in range(self._length)])
    
    def roundIntValues(self):
        return [round(val) for val in self._values]
    
    def roundUp(self):
        return Vector([math.ceil(val) if val >= 0 else math.floor(val) for val in self._values])
    
    def scalarIntDivide(self, div):
        if not isinstance(div, int):
            raise Exception()
        return Vector([val // div for val in self._values])
    
    def toString(self):
        return ', '.join([str(c) for c in self._values])
            

In [3]:
class Matrix:
    
    def __init__(self, matrix):
        self._n = len(matrix)
        self._m = len(matrix[0])
        self._matrix = matrix
        
    def getInverse(self):
        if self._n != self._m:
            raise Exception("Cannot invert a non-square matrix")
        if self._n != 2:
            raise Exception("Not implemented for dimension not equal 2")
        det = self.getDeterminant()
        if det == 0:
            raise Exception("Determinant zero inversion")
        mat = self._matrix
        return Matrix([[  mat[1][1] / det, - mat[0][1] / det], \
                       [- mat[1][0] / det,   mat[0][0] / det]])
        
    def getDeterminant(self):
        if self._n != self._m:
            raise Exception()
        mat = self._matrix
        if self._n == 1:
            return mat[0][0]
        elif self._n == 2:
            return mat[0][0] * mat[1][1] - mat[0][1] * mat[1][0]
        else:
            pdet = 0
            sgn = 1
            for i in range(self._n):
                pdet += sgn * mat[0][i] * self.getMatrixMinor(i).getDeterminant()
                sgn *= -1
            return pdet
    
    def rightMultiplyVector(self, vector):
        if self._m != vector.getLength():
            raise Exception("Matrix dimensions not compatible with vector")
        values = vector.getValues()
        return Vector([sum([self._matrix[i][j] * values[j] for j in range(self._m)]) \
                       for i in range(self._n)])
    
    def getMatrixMinor(self, k):
        if self._n != self._m:
            raise Exception("Matrix minor not implemented for non-square matrices")
        return Matrix([[self._matrix[i + 1][j if j < k else j + 1] \
                        for j in range(self._m - 1)] \
                       for i in range(self._n - 1)])
    
    def getColumnMatrix(*argv):
        length = 0
        for column in argv:
            if length == 0:
                length = column.getLength()
            elif column.getLength() != length:
                raise Exception()
        if length == 0:
            raise Exception()
        return Matrix([[col.getValues()[i] for col in argv] \
                       for i in range(length)])
    
    def invertAndRightMultiplyIntVector(self, vector):
        mat = self._matrix
        pinv = Matrix([[  mat[1][1], - mat[0][1]], \
                       [- mat[1][0],   mat[0][0]]])
        return pinv.rightMultiplyVector(vector).scalarIntDivide(self.getDeterminant())
    
    def toString(self):
        return '\n'.join(['\t'.join([str(el) for el in self._matrix[i]]) for i in range(self._n)])

In [4]:
class Projectile:
    
    def __init__(self, line):
        self._initialPosition, self._velocity = self.initArrays(line)
        
    def initArrays(self, line):
        return [Vector([int(el) for el in part.split(',')]) for part in line.split('@')]
        
    def getInitialPosition(self):
        return self._initialPosition
    
    def getVelocity(self):
        return self._velocity
    
    def getPositionAtTime(self, t):
        return self._initialPosition + self._velocity.scalarMultiply(t)
    
    def getDimensions(self):
        return self._initialPosition.getLength()
        
    def getPathIntersection(this, other):
        if not isinstance(this, Projectile) or not isinstance(other, Projectile):
            raise Exception()
        p = this.getVelocity().truncate(2)
        q = other.getVelocity().truncate(2)
        mat = Matrix.getColumnMatrix(-p, q)
        if mat.getDeterminant() == 0:
            return None
        inv = mat.getInverse()
        b = this.getInitialPosition() - other.getInitialPosition()
        b = b.truncate(2)
        return inv.rightMultiplyVector(b)
    
    def toString(self):
        return self._initialPosition.toString() + ' @ ' + self._velocity.toString()

In [5]:
class Hailstorm:
    
    s_minimumBound = 200000000000000
    s_maximumBound = 400000000000000
    
    def __init__(self, text):
        self._projectiles = self.initProjectiles(text)
        self._length = len(self._projectiles)
        
    def initProjectiles(self, text):
        return [self.makeProjectile(line) for line in text.split('\n')]
    
    def makeProjectile(self, line):
        return Projectile(line)
    
    def getProjectile(self, i):
        return self._projectiles[i]
    
    def getLength(self):
        return self._length
    
    def iterateProjectiles(self):
        for projectile in self._projectiles:
            yield projectile
            
    def iterateProjectilePairs(self):
        for i in range(1, len(self._projectiles)):
            for j in range(i):
                yield self._projectiles[i], self._projectiles[j]
    
    def getCollisionCount(self):
        count = 0
        for i in range(1, len(self._projectiles)):
            for j in range(0, i):
                if Hailstorm.isCollisionInBounds(self._projectiles[i], self._projectiles[j]):
                    count += 1
        return count
                
    def isCollisionInBounds(p, q):
        t = Projectile.getPathIntersection(p, q)
        if t == None:
            return False
        times = t.getValues()
        length = 2
        projectiles = [p, q]
        for i in range(length):
            if times[i] < 0:
                return False
            if not Hailstorm.isWithinBounds(projectiles[i].getPositionAtTime(times[i])):
                return False
        return True
        
    def isWithinBounds(vector):
        values = vector.truncate(2).getValues()
        for v in values:
            if v < Hailstorm.s_minimumBound or Hailstorm.s_maximumBound < v:
                return False
        return True

In [6]:
hailstorm = Hailstorm(text)
hailstorm.getCollisionCount()

16050

In [7]:
class HailProjectile(Projectile):
    
    def __init__(self, line):
        super().__init__(line)
        self._collisionTime = None
        
    def setCollisionTime(self, time):
        if self._collisionTime != None and time != self._collisionTime:
            raise Exception()
        self._collisionTime = time
        
    def getCollisionTime(self):
        return self._collisionTime

In [8]:
class HailstormProjectile(Hailstorm):
    
#     s_positionScaleReductionFactor = 1e0
    s_positionScaleReductionFactor = 1e-12
#     s_initialVelocityValues = [0, 0, 0] 
    s_initialVelocityValues = [213, -167, 248]
#     s_velocityScale = 1e0
    s_velocityScale = 1e2
    s_velocityValues = None
#     s_velocityValues = [-3, 1, 2]
#     s_velocityValues = [214, -168, 249]
    
    def __init__(self, text):
        super().__init__(text)
        self._dimensions = self.getProjectile(0).getDimensions()
    
    #Override
    def makeProjectile(self, line):
        return HailProjectile(line)
        
    def getProjectileVelocity(self):
        if HailstormProjectile.s_velocityValues != None:
            return Vector(HailstormProjectile.s_velocityValues)
        v = Vector(HailstormProjectile.s_initialVelocityValues)
        dv, mse = self.getDifferentials(v)
        v -= dv
        print(mse, v.toString())
        ct = 1
        while mse > .5 * HailstormProjectile.s_velocityScale:
            dv, mse = self.getDifferentials(v)
            v -= dv
            ct += 1
            print(mse, v.toString())
        if HailstormProjectile.s_velocityValues == None:
            HailstormProjectile.s_velocityValues = v.roundIntValues()
        return Vector(HailstormProjectile.s_velocityValues)
        
    def getDifferentials(self, v):
        dv = Vector(self._dimensions * [0])
        basis = [Vector([1 if i == j else 0 for j in range(self._dimensions)]) \
                 for i in range(self._dimensions)]
        sse = 0
        for projectile_1, projectile_2 in self.iterateProjectilePairs():
            x_1 = projectile_1.getInitialPosition()\
            .scalarMultiply(HailstormProjectile.s_positionScaleReductionFactor)
            v_1 = projectile_1.getVelocity()
            x_2 = projectile_2.getInitialPosition()\
            .scalarMultiply(HailstormProjectile.s_positionScaleReductionFactor)
            v_2 = projectile_2.getVelocity()
            mat = Matrix.getColumnMatrix(x_1 - x_2, v_1 - v, v - v_2)
            det = mat.getDeterminant()
            dv += Vector([2 * det * (Matrix.getColumnMatrix(x_1 - x_2, -basis[j], v - v_2).getDeterminant() \
                                     + Matrix.getColumnMatrix(x_1 - x_2, v_1 - v, basis[j]).getDeterminant()) \
                        for j in range(self._dimensions)])
            sse += det ** 2
        dv = self.reduceVelocityDifferential(dv)
        return dv, math.sqrt(sse) / (.5 * self._length * (self._length + 1))
    
    def reduceVelocityDifferential(self, v):
        f = (HailstormProjectile.s_velocityScale ** 5) * 0.5 * self._length * (self._length + 1)
        return Vector([val / ( f ) for val in v.getValues()])
    
    def countInitialPosition(self):
        initialPosition = self.getProjectileInitialPosition()
        return sum(initialPosition.getValues())
    
    def getProjectileInitialPosition(self):
        velocity = self.getProjectileVelocity()
        initialCollisionProjectile = None
        minTime = None
        length = 2
        count = 0
        for projectiles in self.iterateProjectilePairs():
            velocities = [projectiles[0].getVelocity() - velocity,\
                        velocity - projectiles[1].getVelocity()]
            mat = Matrix([[velocities[i].innerProduct(velocities[j]) \
                           for j in range(length)] \
                          for i in range(length)])
            x = projectiles[1].getInitialPosition() - projectiles[0].getInitialPosition()
            b = Vector([x.innerProduct(velocities[i]) for i in range(length)])
            times = mat.invertAndRightMultiplyIntVector(b).getValues()
            count += 1
            if not self.validateCollision(velocity, projectiles, times):
                raise Exception()
            for i in range(length):
                time = times[i]
                projectiles[i].setCollisionTime(time)
                if minTime == None or time < minTime:
                    minTime = time
                    initialCollisionProjectile = projectiles[i]
        initialPosition = initialCollisionProjectile.getPositionAtTime(minTime) \
        - velocity.scalarMultiply(minTime)
        return initialPosition
    
    def validateCollision(self, velocity, projectiles, times):
        length = 2
        left = projectiles[0].getPositionAtTime(times[0]) + velocity.scalarMultiply(times[1] - times[0])
        right = projectiles[1].getPositionAtTime(times[1])
        if not left.getValues() == right.getValues():
            print(left.getValues(), right.getValues())
            return False
        return True

In [9]:
hailstormProjectile = HailstormProjectile(text)
hailstormProjectile.countInitialPosition()

151.18812263111005 213.12067663321864, -167.11309172009072, 247.97263801796586
132.40268296498587 213.19661638627792, -167.21021784106694, 247.97787725018668
120.37659374510005 213.25024634505635, -167.29294887504506, 247.99981640565076
111.31577499005382 213.2925115901557, -167.36329796274637, 248.03027444204588
103.87027804766537 213.32873254996392, -167.42324741244403, 248.0649534671647
97.46803954551697 213.3614820697607, -167.47456625337495, 248.10153261805758
91.80935867628594 213.39199685057417, -167.5187606965939, 248.13871594593277
86.7110700512369 213.42086849181666, -167.55708179521346, 248.17575272091352
82.05175837888234 213.4483807299841, -167.59055472012838, 248.2121925408263
77.74774453391196 213.47467314633587, -167.6200134727437, 248.2477578256227
73.74003623322642 213.49981994593287, -167.6461341343771, 248.2822755335499
69.98623229257798 213.52386731750792, -167.66946410146218, 248.3156391752859
66.45519852588579 213.54685071007708, -167.69044672368855, 248.34778665

669042940632377